# Dependency

In [12]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION='116.0.5845.96'
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium
pip install webdriver-manager

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [591 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,012 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,389 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,185 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packag

# Crawl

In [13]:
import requests
from bs4 import BeautifulSoup

In [23]:
import random

address = []

for i in range(100000):
    address.append(651000000 + random.randint(1, 15000000))

address.sort()

In [24]:
def fetch_data(ad):
    webpage = requests.get("https://www.daangn.com/articles/" + str(ad))
    soup = BeautifulSoup(webpage.content, "html.parser")
    color_list = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10']

    temperature = None
    for color in color_list:
        try:
            temperature = soup.find('section', {'id': 'article-profile'}).find('dl', {'id': 'temperature-wrap'}).find('dd', {'class': 'text-color-' + color}).text.strip()[0:4]
        except:
            continue

    try:
        title = soup.find('h1', {'id': 'article-title'}).text.strip()
    except:
        title = None

    try:
        category = soup.find('p', {'id': 'article-category'}).text.split('∙')[0].strip()
    except:
        category = None

    try:
        time_posted = soup.find('p', {'id': 'article-category'}).find('time').text.strip()
    except:
        time_posted = None

    try:
        price = soup.find('p', {'id': 'article-price'}).text.strip()
    except:
        price = None

    try:
        description = soup.find('div', {'id': 'article-detail'}).text.strip()
    except:
        description = None

    try:
        interest, chat, views = [text.split()[1] for text in soup.find('p', {'id': 'article-counts'}).text.split('∙')]
    except:
        interest, chat, views = None, None, None

    try:
        return {
            'Temperature': temperature,
            'Title': title,
            'Category': category,
            'Time Posted': time_posted,
            'Price': price,
            'Description': description,
            'Interest Count': interest,
            'Chat Count': chat,
            'View Count': views
        }
    except:
        return None

In [25]:
%cd '/content/drive/MyDrive'

/content/drive/MyDrive


In [28]:
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

data = []

with tqdm(total=len(address), desc="Scrapping Progress") as pbar:
    with ThreadPoolExecutor(max_workers=100) as executor:
        results = executor.map(fetch_data, address)
        for result in results:
            if result:
                data.append(result)
            pbar.update(1)

df = pd.DataFrame(data)
df.to_csv("daangn.csv", mode='w', index=False, encoding='utf-8-sig')
df

Scrapping Progress: 100%|██████████| 100000/100000 [2:11:19<00:00, 12.69it/s]


,Temperature,Title,Category,Time Posted,Price,Description,Interest Count,Chat Count,View Count
0,40.5,레고 플레이매트 초록,유아동,끌올 20시간 전,"20,000원",새거 정품 약 90cm입니다\n노랑은 매진이고 초록 가능합니다 ㅎ,12,1,182
1,42.2,볼링,스포츠/레저,끌올 27일 전,"20,000원",볼링가방-1\n아대-1\n신발265m-1\n볼-2\n몽땅입니다\n사용감있습니다,0,2,55
2,38.0,세터 울 부클 니트,남성패션/잡화,끌올 7일 전,"86,000원",많이 입지 않고 장롱에 박혀있던 옷입니다.,1,1,111
3,None,None,None,None,None,None,None,None,None
4,43.0,스트레칭 보드 발목 경사대(6단계),스포츠/레저,끌올 9일 전,"15,000원","발목, 아킬레스 스트레칭 도구입니다.\n모서리에 깨짐이 좀 되어 있습니다.\n무게가...",8,1,107
...,...,...,...,...,...,...,...,...,...
99995,38.6,데이비드 데이비드ez2 4번우드 16.5도 sr .Rafare5번 우드 판매합니다.,스포츠/레저,22시간 전,"80,000원","데이비드ez2 4번우드 16.5도 sr\n판매합니다. 인터넷 158,000에 판매...",0,0,15
99996,56.7,루이비통 앗치,여성잡화,22시간 전,"100,000원","상태 좋아요\n앗치 가방입니다. 정품문의 사절.\nA급이고, 더스트백 있습니다.\...",3,2,209
99997,53.3,"철제 파티션, 발터 파티션 다수 판매합니다",가구/인테리어,22시간 전,"80,000원",여러개있고 모두 상태 좋습니다~\n깨끗하게 사용했어요!\n불투명 유리에요^^\n개당...,17,7,313
99998,39.3,완자 물2 화2,도서,22시간 전,"8,000원",1권당 8000원입니다\n2권 일괄시 15000원에 드려요\n아예 새책입니다\n직거...,0,0,4


In [29]:
df.isnull().sum()

Temperature       32670
Title             33621
Category          33621
Time Posted       33621
Price             38691
Description       32670
Interest Count    33621
Chat Count        33621
View Count        33621
dtype: int64

In [ ]:
# print("Temperature:", temperature)
# print("Title:", title)
# print("Category:", category)
# print("Time Posted:", time_posted)
# print("Price:", price)
# print("Description:", description)
# print("Interest Count:", interest)
# print("Chat Count:", chat)
# print("View Count:", views)

# Test

In [ ]:
webpage  = requests.get("https://www.daangn.com/region/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C/%EA%B0%95%EB%82%A8%EA%B5%AC/%EC%97%AD%EC%82%BC2%EB%8F%99")
soup = BeautifulSoup(webpage.content, "html.parser")

getItem = soup.select("#content > section.cards-wrap > article")

for item in getItem:
    print(item.select('a > div.card-desc > h2 ')[0].text.strip(), end=",")
    print(item.select('a > div.card-desc > div.card-price')[0].text.strip(),end=",")
    print(item.select('a > div.card-desc > div.card-region-name')[0].text.strip(),end=",")
    print(item.select('a > div.card-desc > div.card-counts')[0].text.strip().replace(' ', '').replace('\n', '').replace('∙', ','))

print(item.select('a > div.card-desc > h2 ')[0].text.strip(), end=",")

아디다스 경량패딩,30,000원,서울 강남구 역삼2동,관심16,채팅3
버버리 여성 퀼팅코트 [새상품],800,000원,서울 강남구 역삼2동,관심18,채팅0
ECCO 에코 골프화,40,000원,서울 강남구 역삼2동,관심6,채팅1
삼성 냉장고,100,000원,서울 강남구 역삼2동,관심3,채팅1
네셔널지오그래픽 후리스,45,000원,서울 강남구 역삼2동,관심6,채팅1
아동 스키 장비 풀세트,300,000원,서울 강남구 역삼2동,관심20,채팅1
앤어더스토리스 후드집업,20,000원,서울 강남구 역삼2동,관심5,채팅1
아이폰 5s 실버 팝니다,65,000원,서울 강남구 역삼2동,관심5,채팅3
아일랜드식탁 의자 바의자,35,000원,서울 강남구 역삼2동,관심18,채팅3
나이키모자 헤리티지86 퓨추라 캡모자 블랙 화이트 새상품 정품,18,900원,서울 강남구 역삼2동,관심10,채팅5
크레모아 쓰리페이스 플러스 M,80,000원,서울 강남구 역삼2동,관심7,채팅2
르꼬끄 후리스,35,000원,서울 강남구 역삼2동,관심8,채팅3
키르시 니트,10,000원,서울 강남구 역삼2동,관심1,채팅1
삼성에어컨,130,000원,서울 강남구 역삼2동,관심1,채팅2
글램온 갈바닉,30,000원,서울 강남구 역삼2동,관심2,채팅2
퍼자켓 새상품,99,000원,서울 강남구 역삼2동,관심17,채팅0
까르띠에 러브팔찌,470만원,서울 강남구 역삼2동,관심14,채팅0
손정완 램스킨 + 세이블모피  55시이즈,270,000원,서울 강남구 역삼2동,관심7,채팅0
LPGA투어 골프웨어 방풍패딩자켓 90사이즈,60,000원,서울 강남구 역삼2동,관심7,채팅0
화분,90,000원,서울 강남구 역삼2동,관심4,채팅0
샤넬 카드지갑 똑딱이 은장,480,000원,서울 강남구 역삼2동,관심20,채팅0
샤넬 카드지갑 똑딱이 은장,

In [ ]:
from bs4 import BeautifulSoup

# Example HTML content (replace this with your actual HTML content)
html_content = '''
<article id="content" data-id="665146662">
    <h1 class="hide">폴로 랄프로렌 니트 XS</h1>
    <section id="article-profile">
        <a id="article-profile-link" href="u/A2zCZEKVL5N7lePY">
              <div class="space-between">
                   <dl id="temperature-wrap">
                        <dt>매너온도</dt>
                        <dd class="text-color-03 ">
                             " 37. 1 "
                        </dd>
                   </dl>
              </div>
        </a>
    </section>
</article>
'''

# Parse the HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Select the temperature element
getItem = soup.select("section#article-profile a#article-profile-link dl#temperature-wrap dd.text-color-03")

# Check if the element is found
if getItem:
    temperature = getItem[0].text.strip()
    print(temperature)
else:
    print("Temperature element not found")


" 37. 1 "


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

URL = "https://www.daangn.com/region/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C/%EA%B0%95%EB%82%A8%EA%B5%AC/%EB%8F%84%EA%B3%A1%EB%8F%99"

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=chrome_options)

driver.get(URL)
title = driver.title
driver.close()

print(title)

서울특별시 강남구 도곡동 중고거래 | 당신 근처의 당근


In [1]:
import pandas as pd

In [7]:
import chardet

with open('daangn.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

print(result)

{'encoding': None, 'confidence': 0.0, 'language': None}


In [11]:
df = pd.read_csv('daangn.csv', encoding='utf-8', encoding_errors='ignore')
df[200:210]

<ipython-input-11-b674aa9f1f3a>:1: DtypeWarning: Columns (17,18,19,20,21,109) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('daangn.csv', encoding='utf-8', encoding_errors='ignore')


,Unnamed: 0,Temperature,Title,Category,Time Posted,Price,Description,Interest Count,Chat Count,View Count,...,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112
200,235,52.2,?라 Zara ?즈 골덴바? 2?-10?이?140cm,"?아??30????""7",000??,"블랙, 베이지 골덴바? ?니???리 조절 가?하?거의 ?? ?아???태??최상?예?...",000??,?프?더 블라?스?니??\n?매???릴+고무??테?이 ?어???성?럽????러?리지...,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,237,42.5,가글,?활/주방,"30????""3",000??,?실? ?이 맞??아?요;;;,1,1,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,239,37.4,카메???수진주?찌,?성?화,"?올 28????""57",000??,카메???수진주?찌 ?상???니??카메??보시?최상??니???배 기능?며 직거??부...,3,0,299,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,241,39.1,르꼬르망 ?닝?랫 ?오?드 ?웨?드 210,"?아??30????""15",000??,르꼬르망 ?닝?랫\n?제?이?요. ?래 보??서 ?자 ?벨 ?바램??좀 ???박스?...,3,1,64,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,242,59.3,[?상?? ????리 ?리고당 700g,"가공식??30????""2",000??,?통기한 11??21??까??요.\n?비??문고리거???해??\n?️교환/반품 불?...,000??,?름 = ?좌\n**?피블루 ?으??려??\n?기 = 길이 ??35cm (가???3...,000??,깨끗?고 ?상?동 ?태좋아???자????료 발안공단??6 ?룸?????성??니???...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,247,59.8,립살리스 ?분 :),?물,"?올 3????""15",000??,립살리스 ?분?에??:)\n?분 ?리?면???놓?니?? \n???도 많이 ?고 건강...,17,0,319,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,249,44.1,?활공작???드?시 리필 ?과???활/주방,"?올 4?간 ??""3",000??,리필1개에?? 2024??3??까??니??2,0,45,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
